In [2]:
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np
import pandas as pd
import os
import sys
import rasterio

In [116]:

gisdb = os.path.join(os.path.expanduser("~"), "Documents")
location = "nc_spm_08"
mapset = "PERMANENT"
gisbase='/Applications/GRASS-7.6.app/Contents/Resources'
os.environ['GISBASE'] = gisbase
grass_pydir = os.path.join(gisbase, "etc", "python")
sys.path.append(grass_pydir)
os.environ['LD_LIBRARY_PATH']="/Applications/GRASS-7.6.app/Contents/Resources/lib"
import grass.script as gscript
import grass.script.setup as gsetup


def grass_union(v1_dir,v2_dir,out_dir,patch_n,path_out):
    gsetup.init(gisbase, gisdb, location, mapset)

    gscript.run_command("g.proj",flags="c" ,proj4="+proj=longlat +datum=WGS84 +no_defs")

    gscript.run_command("v.in.ogr", 
    min_area=0.0001 ,
    snap=-1.0, 
    input=v1_dir, 
    output="vector3", 
    overwrite=True, 
    flags="o")

    gscript.run_command("v.in.ogr", 
    min_area=0.0001 ,
    snap=-1.0, 
    input=v2_dir, 
    output="vector4", 
    overwrite=True, 
    flags="o")

    raster=rasterio.open(path_out+'/predicted_tiff/patch'+str(patch_n)+'/merged_prediction.tiff')
    df=raster.bounds

    gscript.run_command("g.region",
    n=df.top,
    s=df.bottom,
    e=df.right,
    w=df.left)

    gscript.run_command("v.overlay", 
    overwrite=True,
    ainput="vector3",
    atype="area", 
    binput="vector4", 
    btype="area", 
    operator='or', 
    snap=0, 
    output="output_b")

    out_file='converted_temp.geojson'
    gscript.run_command("v.out.ogr",
    type="auto",
    input="output_b",
    output=out_file,
    format="GeoJSON",
    overwrite=True)


    temp=gpd.read_file(out_file)
    os.remove(out_file)
    temp.crs={'init' :'epsg:4326'}
    temp.to_file(out_dir)
    
    
def grass_intersect(v1_dir,v2_dir,out_dir,patch_n,path_out):
    gsetup.init(gisbase, gisdb, location, mapset)

    gscript.run_command("g.proj",flags="c" ,proj4="+proj=longlat +datum=WGS84 +no_defs")

    gscript.run_command("v.in.ogr", 
    min_area=0.0001 ,
    snap=-1.0, 
    input=v1_dir, 
    output="vector3", 
    overwrite=True, 
    flags="o")

    gscript.run_command("v.in.ogr", 
    min_area=0.0001 ,
    snap=-1.0, 
    input=v2_dir, 
    output="vector4", 
    overwrite=True, 
    flags="o")

    raster=rasterio.open(path_out+'/predicted_tiff/patch'+str(patch_n)+'/merged_prediction.tiff')
    df=raster.bounds

    gscript.run_command("g.region",
    n=df.top,
    s=df.bottom,
    e=df.right,
    w=df.left)

    gscript.run_command("v.overlay", 
    overwrite=True,
    ainput="vector3",
    atype="area", 
    binput="vector4", 
    btype="area", 
    operator='and', 
    snap=0, 
    output="output_b")

    out_file='converted_temp.geojson'
    gscript.run_command("v.out.ogr",
    type="auto",
    input="output_b",
    output=out_file,
    format="GeoJSON",
    overwrite=True)


    temp=gpd.read_file(out_file)
    os.remove(out_file)
    temp.crs={'init' :'epsg:4326'}
    temp.to_file(out_dir)

In [117]:
box=gpd.read_file('London/shape_box0/shape_box0.shp')

In [118]:
xmin,ymin,xmax,ymax = box.total_bounds

In [119]:
xn = 30
yn = 20

cols = list(np.linspace(xmin, xmax, xn, endpoint=True))
rows = list(np.linspace(ymin, ymax, yn, endpoint=True))
rows.reverse()

polygons = []
X,Y=np.meshgrid(cols,rows)
for x in range(xn-1):
    for y in range(yn-1):
        polygons.append(
            Polygon([(X[y,x], Y[y,x]), (X[y+1,x], Y[y+1,x]), (X[y+1,x+1],Y[y+1,x+1]), (X[y,x+1],Y[y,x+1])]))

grid = gpd.GeoDataFrame({'geometry': polygons})
grid.crs = box.crs
grid['grid_num']=grid.index
grid['grid_area']=grid.area


In [385]:
path_out='London'
patch_n=0
grid.to_file(path_out+'/grid'+str(patch_n))

In [386]:
v1_dir=path_out+'/predict_GUF_roads_mod'+str(patch_n)+'/predict_GUF_roads_mod'+str(patch_n)+'.shp'
v2_dir=path_out+'/grid'+str(patch_n)+'/grid'+str(patch_n)+'.shp'
out_dir=path_out+'/grid_intersect'+str(patch_n)
grass_intersect(v1_dir,v2_dir,out_dir,patch_n,path_out)

In [387]:
grid_intersect=gpd.read_file(out_dir)
grid_intersect=grid_intersect.drop(['cat','a_cat','b_cat'],axis=1)

In [388]:
grid_intersect['area']=grid_intersect.area

In [389]:
temp=grid_intersect.groupby(['b_grid_num','a_LC']).sum()
temp.b_grid_are=grid_intersect.b_grid_are.iloc[0]

In [390]:
temp['percentage']=temp.area/temp.b_grid_are
temp=temp.reset_index()

In [391]:
list_LC=gpd.read_file(v1_dir).LC.unique()

In [392]:
centroids=grid.centroid

In [481]:
fraction=pd.DataFrame(columns=np.concatenate((['lat','lon'],list_LC)),index=centroids.index)
for i in temp.b_grid_num.unique():
     
    a_list=[]
    for x in list_LC:
        try:
            a=temp[(temp.b_grid_num==i)&(temp.a_LC==x)].percentage.values[0]
        except:
            a=0
        a_list.append(a)
        
    aa=np.array(a_list)
    for j in aa:
        dy=(1-np.sum(a_list))/len(aa[aa!=0])
        if j!=0:
            aa[aa==j]=j+dy
    a_list=aa.T.tolist()
    
    fraction.loc[i]=[centroids.loc[i].y,centroids.loc[i].x]+a_list

            

In [ ]:
fraction.to_csv(path_out+'/fraction'+str(patch_n)+'.csv')

In [115]:
import geopandas as gpd
from pyproj import Proj, transform
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import xarray as xr
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colors as colors

In [304]:
counter=0
for i in range(0,12):
    counter+=1
    if counter==1:
        df=pd.read_csv('London/fraction'+str(i)+'.csv',index_col=0)
    else:
        df1=pd.read_csv('London/fraction'+str(i)+'.csv',index_col=0)
        df=pd.concat([df,df1])
df.lon=df.lon.round(6)
df.lat=df.lat.round(6)
X,Y=np.meshgrid(np.sort(df.lon.unique()),np.sort(df.lat.unique()))
Z=X*0

In [309]:
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        Z[i,j]=df[(df.lat==Y[i,j])&(df.lon==X[i,j])]['0.0']

In [ ]:
plt.subplots(1,figsize=(10,8))
plt.pcolor(X,Y,Z)
plt.colorbar()

In [ ]:
np.mean(Z)

# Calculating Fraction

In [303]:
from fraction_util import calculate_fraction
path_out='London'
xn=50
yn=50
for patch_n in range(1,12):
        calculate_fraction(path_out,patch_n,xn,yn)

calculating the land cover fraction . . .
calculating the land cover fraction . . .
calculating the land cover fraction . . .
calculating the land cover fraction . . .
calculating the land cover fraction . . .
calculating the land cover fraction . . .
calculating the land cover fraction . . .
calculating the land cover fraction . . .
calculating the land cover fraction . . .
calculating the land cover fraction . . .
calculating the land cover fraction . . .
